# 🗺️ Traffic Speed Visualization (MapboxGL)

In [4]:
# 📦 Install dependencies (if not already installed)
!pip install requests pandas geopandas mapboxgl shapely --quiet

In [15]:
# 🔧 Setup
import requests
import pandas as pd
from mapboxgl.viz import ChoroplethViz
from mapboxgl.utils import create_color_stops

# Replace with your actual Mapbox token
MAPBOX_TOKEN = "pk.eyJ1IjoiemVuY2hhcmxpZSIsImEiOiJjbWNoMHJuNmgwbzR2Mm5vdDhncjZtOGczIn0.P0j16UZhaq-nh3iNF-NX9Q"
#viz.show()
# API base URL
BASE_URL = "http://127.0.0.1:8000"


In [ ]:
# 🧪 Query aggregated API endpoint
BASE_URL = "http://127.0.0.1:8000"
params = {
    "day_of_week": "Monday",
    "time_period": "AM Peak"
}
response = requests.get(f"{BASE_URL}/aggregates/", params=params)
# Optional debug
print(response.status_code)
print(response.text)
# GeoJSON-style features
geojson_data = response.json()
print(f"Retrieved {len(geojson_data)} features")

In [ ]:
# 🎨 Build color-coded map
features = [
    {
        "type": "Feature",
        "geometry": f["geometry"],
        "properties": {
            "average_speed": f["properties"]["average_speed"],
            "road_name": f["properties"].get("road_name", "Unnamed")
        }
    }
    for f in geojson_data
]

viz = ChoroplethViz(
    {
        "type": "FeatureCollection",
        "features": features
    },
    access_token=MAPBOX_TOKEN,
    color_property="average_speed",
    color_stops=create_color_stops([10, 20, 30, 40, 50], colors='Reds'),
    center=(-81.6557, 30.3322),
    zoom=11,
    line_width=1.5,
    opacity=0.8,
    legend_title="Average Speed (mph)"
)
viz.show()

In [ ]:
# 📊 Optional Tabular Summary
df = pd.DataFrame([
    {
        "road_name": f["properties"].get("road_name", "Unnamed"),
        "avg_speed": f["properties"]["average_speed"]
    }
    for f in geojson_data
])

df.sort_values("avg_speed").head(10)